In [1]:
from __future__ import print_function

import cellconstructor as CC, cellconstructor.Phonons
import sscha, sscha.Ensemble

import tdscha, tdscha.DynamicalLanczos as DL
import time, numpy as np
import matplotlib.pyplot as plt
import nonlinear_sscha.Conversion as conv

ModuleNotFoundError: No module named 'tdscha.DynamicalLanczos'; 'tdscha' is not a package

## Spectral function

In [ ]:
# Here the input of the TDSCHA calculation
USE_WIGNER = True
# Lanczos status file
if not USE_WIGNER:
    DATA_FILE = "Lanczos_H2/tdscha_lanczos_STEP2.npz"
else:
    DATA_FILE = "Lanczos_H2_wigner/tdscha_lanczos_STEP3.npz"

# If you submitted with the tdscha-lanczos.x
# Copy all the files inside the directory
LOAD_FROM_EXECUTABLE = False
DIRECTORY = "submit_on_cluster"
PREFIX = "tdscha_lanczos"
   
# INPUT VARIABLES -------------------
FREQ_START = 0 # cm-1
FREQ_END = 1000 # cm-1
N_FREQS = 10000

# Load the lanczos
lanczos = DL.Lanczos()

if not LOAD_FROM_EXECUTABLE:
    lanczos.load_status(DATA_FILE)
else:
    lanczos.load_from_input_files(PREFIX, DIRECTORY)

In [ ]:
# Print on the screen the static value of the free energy hessian
# (In this way we show how to compute the static free energy hessian).
w_static  = np.array([0])
gf_static = lanczos.get_green_function_continued_fraction(w_static)

# The free energy hessian is the inverse of the static green function (real part)
# computed at omega = 0
hessian = 1 / np.real(gf_static[0])

# Get the frequency (the square root with negative sign for imaginary values)
static_frequency = np.sign(hessian) * np.sqrt(np.abs(hessian))

# Convert from Ry to cm-1
static_frequency *= CC.Units.RY_TO_CM

print("The static frequency of the simualated mode is: {} cm-1".format(static_frequency))

In [ ]:
# Get the frequency array for plotting
w = np.linspace(FREQ_START, FREQ_END, N_FREQS)

# Convert in Ry
w_ry = w / CC.Units.RY_TO_CM

# Get the dynamical green function
green_function = lanczos.get_green_function_continued_fraction(w_ry)

# Plot the imaginary part
plt.figure(dpi = 150)
plt.xlabel("Frequency [cm-1]")
plt.ylabel("- Im(G)")
plt.title("Spectral function")

plt.plot(w, -np.imag(green_function), label = '-Im(G)')
plt.plot(w, np.real(green_function), label = 'Re(G)')
# plt.vlines(-static_frequency, np.max([np.max(-np.imag(green_function)),np.max(np.real(green_function))]),\
#            np.min([np.min( -np.imag(green_function)),np.min(np.real(green_function))]), ls = '--', color = 'green', label = 'static frequency')
plt.legend(loc='best')
plt.tight_layout()
# if not USE_WIGNER:
#     plt.savefig('normal.eps')
# else:
#     plt.savefig('wigner.eps')

In [ ]:
# if not USE_WIGNER:
#     np.save('ReG',+np.real(green_function))
#     np.save('ImG',-np.imag(green_function))
# else:
#     np.save('Wigner_ReG',np.real(green_function))
#     np.save('Wigner_ImG',-np.imag(green_function))

In [ ]:
# ReG   = np.load('ReG.npy')
# W_ReG = np.load('Wigner_ReG.npy')

# plt.plot(w,ReG*1e-3,label = 'Normal')
# plt.plot(w,-W_ReG, label = 'Wigner')
# plt.legend()
# plt.tight_layout()
# plt.savefig('ReG_comp.eps')

In [ ]:
# ImG   = np.load('ImG.npy')
# W_ImG = np.load('Wigner_ImG.npy')

# plt.plot(w,ImG, label = 'Normal')
# plt.plot(w,W_ImG, label = 'Wigner')
# plt.legend()
# plt.tight_layout()
# plt.savefig('ImG_comp.eps')